In [1]:
import os
import random

In [2]:
cd ..

/kaggle


In [3]:
cd ..


/


In [4]:
ls

'=24.4'                                     libx32@
 NGC-DL-CONTAINER-LICENSE                   media/
 bin@                                       mnt/
 boot/                                      opt/
 cuda-keyring_1.0-1_all.deb                 proc/
 dev/                                       root/
 entrypoint.sh*                             run/
 etc/                                       run_jupyter.sh*
 home/                                      sbin@
 install_packages.sh*                       srv/
 kaggle/                                    sys/
 lib@                                       tmp/
 lib32@                                     usr/
 lib64@                                     var/
 libnvinfer8_8.6.1.6-1+cuda12.0_amd64.deb


In [5]:
### read words ###
text_file = open('kaggle/input/dataset/words_250000_train.txt',"r")
full_dictionary = text_file.read().splitlines()


In [6]:
cd /kaggle/working

/kaggle/working


In [ ]:
import numpy as np

random.seed(40)

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

random.seed(40)

# Sample data for demonstration
words = full_dictionary

def generate_training_data_3(words):
    X = []
    y = []
    for word in words:
      # print(word)
      for char in np.unique(list(word)):
          X.append(word.replace(char,"_"))  # Input with missing character
          y.append(char)
          try:
            X.append(word.replace(char,"_").replace(random.choice(list(word.replace(char,""))),"_"))  # Input with missing character
            y.append(char)                      # Target character (the missing one)
          except:
            pass
          # print(word.replace(char,"_"))
    return X, y


# Toy dataset
X_train_3, y_train_3 = generate_training_data_3(words)

words_3 = X_train_3
labels_3 = y_train_3

max_length_3 = max(len(word) for word in words_3)

# Create a dictionary for character to index mapping
char_to_index = {char: idx for idx, char in enumerate(set(''.join(words_3)))}
index_to_char = {idx: char for char, idx in char_to_index.items()}

# Convert words and labels to numerical format
X_padded_3 = pad_sequences([[char_to_index[char] for char in word] for word in words_3], maxlen=max_length_3, padding='post')
y_padded_3 = np.array([char_to_index[label] for label in labels_3])

X_train_3, X_temp_3, y_train_3, y_temp_3 = train_test_split(X_padded_3, y_padded_3, test_size=0.3, random_state=42)
X_val_3, X_test_3, y_val_3, y_test_3 = train_test_split(X_temp_3, y_temp_3, test_size=0.5, random_state=42)

# print("Padded data:")
# for i in range(len(words)):
#     print(f"{words[i]} -> {labels[i]}")

# print("\nPadded and numerical data:")
# print(X_padded_2)
# print(y_padded_2)


# BiLSTM with attention

## attention 

## LSTM 256

## embedding - 50

## no dropout

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Define a checkpoint callback
checkpoint_callback_6 = ModelCheckpoint('best_model_6.keras', 
                                      monitor='val_loss', 
                                      save_best_only=True, 
                                      mode='min', 
                                      verbose=1)

early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                       patience=3, 
                                       mode='min', 
                                       verbose=1)

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_6 = Sequential()

# Add layers to the model
model_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_6.add(AttentionLayer())

# Output layer
model_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_6.summary()


In [ ]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=3, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

In [ ]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=7, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

In [ ]:
# checkpoint_callback_total6 = ModelCheckpoint('best_model_6.keras', 
#                                       monitor='val_loss', 
#                                       save_best_only=True, 
#                                       mode='min', 
#                                       verbose=1)

In [ ]:
# Train the model
model_6.fit(X_train_3, 
            y_train_3, 
            epochs=3,      
            batch_size=256,                     
            validation_data=(X_val_3, y_val_3), 
            verbose=1,
            callbacks=[checkpoint_callback_6,early_stopping_callback])

# Complete Training data
# randome shuffling 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_complete_6 = Sequential()

# Add layers to the model
model_complete_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_complete_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_complete_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_complete_6.add(AttentionLayer())

# Output layer
model_complete_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_complete_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_complete_6.summary()


In [ ]:
from sklearn.utils import shuffle

X_train_complete_3, y_train_complete_3 = shuffle( X_padded_3, y_padded_3 )

# Train the model
model_complete_6.fit( X_train_complete_3, 
            y_train_complete_3,
            epochs=8, 
            batch_size=256, 
#             validation_data=(X_val_3, y_val_3), 
            verbose=1,
#             callbacks=[checkpoint_callback_6,early_stopping_callback]
           )

In [ ]:
model_complete_6.save('best_model_6_complete.keras')

In [ ]:
model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')
from IPython.display import FileLink
FileLink(r'best_model_6_complete.keras')

In [ ]:
# model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')

In [ ]:
## load model 
# model_6_recon = tf.keras.models.load_model('best_model_6.keras')

In [ ]:
# Evaluate on test set
loss, accuracy = model_complete_6_recon.evaluate(X_test_3, y_test_3, verbose=1)
print(f"validation set Accuracy: {accuracy*100:.2f}%")